# Situação da doença: Valores de referência

Construção dos valores de referência para o indicador de Situação da doença (Novos casos diários - Média móvel 7 dias).

**Atualizado em: 30/08/2020**

----

## Pacotes e configurações

In [12]:
import pandas as pd

# Plotting
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

### Código IBGE das capitais

In [13]:
ibge_capitais = {
    "Rio Branco": 1200401,
    "Maceió": 2704302,
    "Macapá": 1600303,
    "Manaus": 1302603,
    "Salvador": 2927408,
    "Fortaleza": 2304400,
    "Brasília": 5300108,
    "Vitória": 3205309,
    "Goiânia": 5208707,
    "São Luís": 2111300,
    "Cuiabá": 5103403,
    "Campo Grande": 5002704,
    "Belo Horizonte": 3106200,
    "Belém": 1501402,
    "João Pessoa": 2507507,
    "Curitiba": 4106902,
    "Recife": 2611606,
    "Teresina": 2211001,
    "Rio de Janeiro": 3304557,
    "Natal": 2408102,
    "Porto Alegre": 4314902,
    "Porto Velho": 1100205,
    "Boa Vista": 1400100,
    "Florianópolis": 4205407,
    "São Paulo": 3550308,
    "Aracaju": 2800308,
    "Palmas": 1721000
}

## Casos reportados nas capitais

In [26]:
# Casos disponibilizados no Brasil.IO
url = "http://45.55.43.231:7000/br/cities/cases/full"
cases = pd.read_csv(url).assign(last_updated=lambda df: pd.to_datetime(df["last_updated"]))

update = cases["last_updated"].max().strftime("%d/%m/%y")
update

'30/08/20'

In [15]:
# Casos filtrados nas capitais
capitals = cases[cases["city_id"].isin(ibge_capitais.values())].dropna()
capitals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3935 entries, 1914 to 615016
Data columns (total 36 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   active_cases                                   3935 non-null   float64       
 1   city_id                                        3935 non-null   int64         
 2   city_name                                      3935 non-null   object        
 3   confirmed_cases                                3935 non-null   float64       
 4   daily_cases                                    3935 non-null   float64       
 5   daily_cases_growth                             3935 non-null   object        
 6   daily_cases_mavg                               3935 non-null   float64       
 7   daily_cases_mavg_100k                          3935 non-null   float64       
 8   deaths                                         3935 n

In [16]:
capitals["daily_cases_mavg"].describe()

count    3935.000000
mean      246.950368
std       404.453136
min        -8.700000
25%        21.950000
50%       119.000000
75%       286.500000
max      3216.900000
Name: daily_cases_mavg, dtype: float64

## Filtro dos casos

### Remove datas com valores negativos (inconsistências de reporte)

In [17]:
capitals[capitals["daily_cases_mavg"] < 0][["last_updated", "city_id", "city_name", "population", 
                                            "daily_cases", "daily_cases_mavg", "daily_cases_mavg_100k"]]

,last_updated,city_id,city_name,population,daily_cases,daily_cases_mavg,daily_cases_mavg_100k
497039,2020-05-07,4205407,Florianópolis,500973,-4.0,-6.1,-1.217630
497061,2020-05-29,4205407,Florianópolis,500973,3.0,-8.7,-1.736621


In [18]:
capitals = capitals[capitals["daily_cases_mavg"] > 0]

### Distribuição final dos casos 

In [28]:
title = f"Média semanal de novos casos por 100 mil nas capitais (atualizado em: {update})"
capitals.pivot_table(index="last_updated", 
                     columns="city_name", 
                     values="daily_cases_mavg_100k").iplot(kind="scatter", 
                                                           theme="white",
                                                           title=title)

## Definição dos valores: Média dos quartis

In [20]:
import numpy as np

In [29]:
quartis = capitals.groupby("city_name").apply(lambda row: np.quantile(row["daily_cases_mavg_100k"], 
                                                                    np.arange(0, 1, 0.25)))
quartis = pd.DataFrame(quartis.to_list(), index=quartis.index).T

In [30]:
pd.options.display.max_columns = 30
quartis.index.name = "quartis"
quartis.columns.name = None 
quartis

,Aracaju,Belo Horizonte,Belém,Boa Vista,Brasília,Campo Grande,Cuiabá,Curitiba,Florianópolis,Fortaleza,Goiânia,João Pessoa,Macapá,Maceió,Manaus,Natal,Palmas,Porto Alegre,Porto Velho,Recife,Rio Branco,Rio de Janeiro,Salvador,São Luís,São Paulo,Teresina,Vitória
quartis,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.045661,0.171174,0.026796,0.225444,0.063013,0.100448,0.391807,0.212094,0.119767,0.621876,0.059362,0.037082,0.059603,0.039256,0.018325,0.067864,0.100292,0.020219,0.075537,0.103298,0.171855,0.025302,0.073111,0.127055,0.002449,0.046251,0.359020
1,1.689464,0.627968,6.806253,5.535892,1.122620,0.435277,0.718312,0.548341,1.586912,5.728003,0.862403,4.922653,6.576242,4.632229,6.295919,2.296063,3.460069,0.579604,5.575552,8.247115,4.185908,2.431200,2.929660,7.305669,3.201104,3.364765,8.174605
2,22.693615,2.581536,13.980955,28.055199,13.895946,1.674141,6.448485,1.764001,3.453280,9.736482,4.508239,22.267820,12.496846,14.946788,11.659534,13.668928,10.029185,1.536625,31.451589,11.347569,15.405616,5.657173,16.328111,9.275024,11.306704,16.153184,24.413348
3,61.292547,15.090145,18.891371,90.152375,50.779898,23.538419,23.145163,19.864415,10.509548,15.972101,15.381439,33.151425,26.225496,24.711762,15.770150,26.407555,32.444413,7.614046,50.557650,14.206487,23.722193,11.577039,24.556573,16.015751,17.860316,31.947921,38.042293


In [22]:
quartis.mean(axis=1)

quartis
0     0.124591
1     3.697770
2    12.471701
3    27.386240
dtype: float64